In [1]:
!pip install -q transformers peft bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 14.1 MB/s eta 0:00:00


In [2]:
from peft import PeftModel
from transformers import AutoModelForCausalLM

base_model = AutoModelForCausalLM.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype="auto",
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, "/content/adapters")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
model = model.merge_and_unload()
print("MODEL MERGED SUCCESSFULLY")


MODEL MERGED SUCCESSFULLY


In [4]:
model.save_pretrained("/content/quantized/model-fp16")


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
)

tokenizer.save_pretrained("/content/quantized/model-fp16")

print("TOKENIZER SAVED")


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

TOKENIZER SAVED


In [6]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True
)

model_int8 = AutoModelForCausalLM.from_pretrained(
    "/content/quantized/model-fp16",
    quantization_config=bnb_config,
    device_map="auto"
)

model_int8.save_pretrained("/content/quantized/model-int8")

print("INT8 MODEL SAVED")


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

INT8 MODEL SAVED


In [7]:
bnb_config_4 = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

model_int4 = AutoModelForCausalLM.from_pretrained(
    "/content/quantized/model-fp16",
    quantization_config=bnb_config_4,
    device_map="auto"
)

model_int4.save_pretrained("/content/quantized/model-int4")

print("INT4 MODEL SAVED")


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

INT4 MODEL SAVED


In [8]:
!ls /content/quantized


model-fp16  model-int4	model-int8


In [9]:
!git clone https://github.com/ggerganov/llama.cpp


Cloning into 'llama.cpp'...
remote: Enumerating objects: 80748, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 80748 (delta 24), reused 30 (delta 18), pack-reused 80686 (from 2)
Receiving objects: 100% (80748/80748), 298.64 MiB | 40.96 MiB/s, done.
Resolving deltas: 100% (58362/58362), done.


In [10]:
!ls llama.cpp


AGENTS.md		       convert_lora_to_gguf.py	pocs
AUTHORS			       docs			poetry.lock
benches			       examples			pyproject.toml
build-xcframework.sh	       flake.lock		pyrightconfig.json
ci			       flake.nix		README.md
CLAUDE.md		       ggml			requirements
cmake			       gguf-py			requirements.txt
CMakeLists.txt		       grammars			scripts
CMakePresets.json	       include			SECURITY.md
CODEOWNERS		       LICENSE			src
common			       licenses			tests
CONTRIBUTING.md		       Makefile			tools
convert_hf_to_gguf.py	       media			vendor
convert_hf_to_gguf_update.py   models
convert_llama_ggml_to_gguf.py  mypy.ini


In [1]:
!pip install -r /content/llama.cpp/requirements.txt


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment


In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
)

tokenizer.save_pretrained("/content/quantized/model-fp16")

print("TOKENIZER SAVED")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


TOKENIZER SAVED


In [3]:
!python /content/llama.cpp/convert_hf_to_gguf.py \
    /content/quantized/model-fp16 \
    --outfile /content/quantized/model.gguf \
    --outtype q8_0


INFO:hf-to-gguf:Loading model: model-fp16
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:hf-to-gguf:gguf: indexing model part 'model.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:output.weight,               torch.bfloat16 --> Q8_0, shape = {2048, 32000}
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> Q8_0, shape = {2048, 32000}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.bfloat16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.bfloat16 --> Q8_0, shape = {5632, 2048}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.bfloat16 --> Q8_0, shape = {2048, 5632}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.bfloat16 --> Q8_0, shape = {2048, 5632}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.bfloat16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.bfloat16 --> Q8_0, shape = {2048, 256}
INFO:hf-to-gguf:blk.0.at

In [4]:
!pip uninstall -y torchvision torchaudio
!pip install -q transformers accelerate bitsandbytes sentencepiece


Found existing installation: torchvision 0.24.0+cpu
Uninstalling torchvision-0.24.0+cpu:
  Successfully uninstalled torchvision-0.24.0+cpu
Found existing installation: torchaudio 2.9.0+cpu
Uninstalling torchaudio-2.9.0+cpu:
  Successfully uninstalled torchaudio-2.9.0+cpu


In [1]:
import time
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_PATH = "/content/quantized/model-fp16"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    torch_dtype="auto"
)

prompt = "Explain artificial intelligence in simple words."

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

start = time.time()
output = model.generate(**inputs, max_new_tokens=120)
end = time.time()

tokens = output.shape[1] - inputs.input_ids.shape[1]
print("Tokens/sec:", tokens/(end-start))


`torch_dtype` is deprecated! Use `dtype` instead!


Tokens/sec: 0.3928740767800524


In [2]:
!cp /content/quantized/model-fp16/tokenizer* /content/quantized/model-int8/
!cp /content/quantized/model-fp16/special_tokens_map.json /content/quantized/model-int8/

!cp /content/quantized/model-fp16/tokenizer* /content/quantized/model-int4/
!cp /content/quantized/model-fp16/special_tokens_map.json /content/quantized/model-int4/


In [3]:
import time
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_PATH = "/content/quantized/model-int8"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    torch_dtype="auto"
)

prompt = "Explain artificial intelligence in simple words."

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

start = time.time()
output = model.generate(**inputs, max_new_tokens=120)
end = time.time()

tokens = output.shape[1] - inputs.input_ids.shape[1]
print("Tokens/sec:", tokens/(end-start))


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Tokens/sec: 0.0892285767907722


In [4]:
import time
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_PATH = "/content/quantized/model-int4"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    torch_dtype="auto"
)

prompt = "Explain artificial intelligence in simple words."

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

start = time.time()
output = model.generate(**inputs, max_new_tokens=120)
end = time.time()

tokens = output.shape[1] - inputs.input_ids.shape[1]
print("Tokens/sec:", tokens/(end-start))


Tokens/sec: 0.036351618365694054


In [8]:
!git clone https://github.com/ggml-org/llama.cpp


Cloning into 'llama.cpp'...
remote: Enumerating objects: 80748, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 80748 (delta 17), reused 16 (delta 16), pack-reused 80719 (from 2)
Receiving objects: 100% (80748/80748), 298.65 MiB | 32.49 MiB/s, done.
Resolving deltas: 100% (58365/58365), done.
Updating files: 100% (2274/2274), done.


In [9]:
!cmake -B llama.cpp/build -S llama.cpp



-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "2.34.1")
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- GGML_SYSTEM_ARCH: x86
-- Including CPU backend
-- Found OpenMP_C: 

In [10]:
!cmake --build llama.cpp/build --config Release -j4


[  0%] Building CXX object vendor/cpp-httplib/CMakeFiles/cpp-httplib.dir/httplib.cpp.o
[  0%] Building CXX object common/CMakeFiles/build_info.dir/build-info.cpp.o
[  1%] Building C object examples/gguf-hash/CMakeFiles/sha256.dir/deps/sha256/sha256.c.o
[  1%] Building C object ggml/src/CMakeFiles/ggml-base.dir/ggml.c.o
[  1%] Built target build_info
[  1%] Building CXX object ggml/src/CMakeFiles/ggml-base.dir/ggml.cpp.o
[  1%] Built target sha256
[  2%] Building C object examples/gguf-hash/CMakeFiles/xxhash.dir/deps/xxhash/xxhash.c.o
[  3%] Building C object ggml/src/CMakeFiles/ggml-base.dir/ggml-alloc.c.o
[  3%] Building C object examples/gguf-hash/CMakeFiles/sha1.dir/deps/sha1/sha1.c.o
[  3%] Built target sha1
[  3%] Building CXX object tools/mtmd/CMakeFiles/llama-llava-cli.dir/deprecation-warning.cpp.o
[  3%] Built target xxhash
[  4%] Building CXX object tools/mtmd/CMakeFiles/llama-gemma3-cli.dir/deprecation-warning.cpp.o
[  4%] Linking CXX executable ../../bin/llama-llava-cli
[  4

In [11]:
!/content/llama.cpp/build/bin/llama-cli \
  -m /content/quantized/model.gguf \
  -p "Explain artificial intelligence simply" \
  -n 120



Loading model... |-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\| 


▄▄ ▄▄
██ ██
██ ██  ▀▀█▄ ███▄███▄  ▀▀█▄    ▄████ ████▄ ████▄
██ ██ ▄█▀██ ██ ██ ██ ▄█▀██    ██    ██ ██ ██ ██
██ ██ ▀█▄██ ██ ██ ██ ▀█▄██ ██ ▀████ ████▀ ████▀
                                    ██    ██
                                    ▀▀    ▀▀

build      : b8103-03fd9d3bb
model      : model.gguf
modalities : text

available commands:
  /exit or Ctrl+C     stop or exit
  /regen              regenerate the last response
  /clear              clear the chat history
  /read               add a text file


> Explain artificial intelligence simply

|-\|/-\|/-\|/- Artificial intelligence (AI) is a field of computer science that focuses on developing intelligent machines that can perform tasks that would typically require human intelligence, such as reasoning, problem-solving, and decision-making. AI can be divided into two main categories: machine learning a

In [ ]:
!du -sh *


In [ ]:
!pip install -q bitsandbytes>=0.46.1 accelerate transformers sentencepiece


In [ ]:
import torch
import bitsandbytes as bnb

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print("bnb loaded")


In [ ]:
import time
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_PATH = "/content/quantized/model-fp16"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    torch_dtype="auto"
)

prompt = "Explain artificial intelligence in simple words."

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

start = time.time()
output = model.generate(**inputs, max_new_tokens=120)
end = time.time()

tokens = output.shape[1] - inputs.input_ids.shape[1]
print("Tokens/sec:", tokens/(end-start))
